# Copy instead of modifying to preserve attack parameters

In [17]:
AGENT_NAME = '20 bin PPO 500 results/default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SURROGATE_PATH = '20 bin PPO 500 results/surrogates/imitator.pth'
#RUNS = 5
TRIAL = 1
SAVE_DIR = '20 bin PPO 500 results/imitator fgm restarts vs asr' + '/'

In [18]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.estimators.classification import PyTorchClassifier as classifier
from art.attacks.evasion import FastGradientMethod as FGM

import pandas as pd
import numpy as np
import os
import torch
from torch.nn import CrossEntropyLoss

import KBMproject.utilities as utils

from joblib import Parallel, delayed

%matplotlib inline

In [19]:
schema = DataSet.get_schema(DATASET_NAME)

In [20]:
restarts = np.arange(0, 6,dtype='int64')
RUNS = len(restarts)

Define RL agent

In [21]:
agents = []
for _ in range (RUNS):
    agents.append(PPO.load(AGENT_NAME))

Create environments

In [22]:
envs = []
for _ in range (RUNS):
    envs.append(utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42))

In [23]:
cols = utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42).observation_names

In [24]:
imitator = torch.load(SURROGATE_PATH)

In [25]:
observation_masks = np.ones(agents[0].observation_space.shape)
observation_masks[0:6] = 0 #mask time features
print('masked features:')
cols[0][0:6]

masked features:


['month_cos',
 'month_sin',
 'day_type_cos',
 'day_type_sin',
 'hour_cos',
 'hour_sin']

In [26]:
kwargs = dict(norm=np.inf,
              #minimal=True,
              eps=0.2,
              #eps_step=0.1,
              #num_random_init=2,
        )
attacks =[]
for run in range(RUNS):
    attacks.append(FGM(estimator=classifier(
                                        model=imitator,
                                        loss=CrossEntropyLoss(), 
                                        nb_classes=agents[run].action_space[0].n, 
                                        input_shape=agents[run].observation_space.shape,
                                        device_type='gpu',
                                        clip_values = (agents[run].observation_space.low.min(),
                                                       agents[run].observation_space.high.max()),
                                        ),
                        num_random_init=int(restarts[run]), #np.int64 is not an int....
                        **kwargs))

In [27]:
%%time
%%capture
results = Parallel(n_jobs=RUNS, verbose=10, prefer='threads')(delayed(
    utils.eval_untargeted_attack)(agent, env, attack) for agent, env, attack in zip(agents, envs, attacks)) 


CPU times: total: 21min 43s
Wall time: 12min 45s


Results is a list of tupples for each run, of the format(KPIs, observations, perturbed observations, epsilons)

In [28]:
kpis = [results[i][0] for i in range(len(results))]
df_kpis = pd.concat(kpis, axis='columns',keys=restarts)

In [29]:
#df_kpis[['mean', 'std', 'variance']] = df_kpis.agg(['mean','std', 'var'], axis='columns')

In [30]:
df_kpis

,0,1,2,3,4,5
cost_function,,,,,,
annual_peak_average,1.214377,1.347728,1.128626,1.120856,1.234943,1.128626
carbon_emissions_total,0.980023,0.989053,0.994134,1.008053,1.005664,1.001577
cost_total,0.920987,0.931437,0.939575,0.956499,0.950518,0.945492
daily_one_minus_load_factor_average,1.024081,0.993278,0.982407,0.992717,1.008913,0.989586
daily_peak_average,1.187960,1.186423,1.176559,1.247385,1.226911,1.211588
electricity_consumption_total,0.980652,0.990049,0.995474,1.009732,1.004492,1.000062
monthly_one_minus_load_factor_average,1.007598,1.002795,0.996126,0.992103,1.010963,1.000502
ramping_average,1.676788,1.725470,1.742659,1.835353,1.800903,1.765366
zero_net_energy,1.123619,1.128610,1.129924,1.133969,1.131727,1.131166


In [31]:
df_kpis.to_csv(SAVE_DIR + f'{RUNS} run KPIs {TRIAL}.csv')

FGSM using the same parameters in a whitebox attack has an ASR of 0.7.

Increasing the number of restarts decreases the ASR, likely because of differences between the surrogate and vicitm models

In [32]:
ASRs = [results[i][3] for i in range(len(results))]
#print(f'For 5 runs of the random attack \nthe mean ASR is: {np.mean(ASRs):.3f}\nthe STD is: {np.std(ASRs):.3f}')

In [33]:
pd.DataFrame({'ASRs':ASRs}, index=restarts,).to_csv(SAVE_DIR + 'ASR for varied epsilons.csv')

In [35]:
ASRs

[0.8326292955816874,
 0.8218974768809225,
 0.8255508619705446,
 0.8302317616166229,
 0.8352551661148533,
 0.8343418198424478]

In [34]:
for run in range(RUNS):
        pd.DataFrame(results[run][1],columns=cols,).to_csv(SAVE_DIR+f'run {run} obs {TRIAL}.csv',)
        pd.DataFrame(results[run][2],columns=cols,).to_csv(SAVE_DIR+f'run {run} adv obs {TRIAL}.csv')